In [1]:
import sys
import os

# Adiciona o diretório pai ao caminho de pesquisa de módulos
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
import wandb
import torchvision.models as models

import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import lr_scheduler

import gc
import random
import pickle
from copy import deepcopy
import numpy as np

In [3]:
from torchvision import transforms

from tqdm import tqdm
from avalanche.benchmarks.classic import SplitCUB200
from utils import create_plot_images
from torch.optim.lr_scheduler import ReduceLROnPlateau

from avalanche.evaluation.metrics import ConfusionMatrix, WandBStreamConfusionMatrix
from avalanche.evaluation.metric_utils import default_cm_image_creator

In [4]:
train_transform = transforms.Compose(
    [
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

eval_transform = transforms.Compose(
    [
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

In [5]:
dataset_name = 'cub-200'
total_number_classes = 200
n_split_experiences = 5

# creating the benchmark (scenario object)
split_ds = SplitCUB200(
    n_experiences=n_split_experiences,
    seed=1234,
    train_transform=train_transform,
    eval_transform=eval_transform
    
)

train_stream = split_ds.train_stream
test_stream = split_ds.test_stream

train_ds = []
test_ds = []
masks = []
classes_in_experience = []

for experience in train_stream:
    print("Start of task ", experience.task_label, 'experience.current_experience', experience.current_experience)
    print('Classes in this task:', experience.classes_in_this_experience)
    masks.append([i for i in range(total_number_classes) if i not in experience.classes_in_this_experience])
    classes_in_experience.append(experience.classes_in_this_experience)
    
    # The current Pytorch training set can be easily recovered through the 
    # experience
    current_training_set = experience.dataset
    train_ds.append(current_training_set)
    
    # ...as well as the task_label
    print('Task {}'.format(experience.task_label))
    print('This task contains', len(current_training_set), 'training examples')

    # we can recover the corresponding test experience in the test stream
    current_test_set = test_stream[experience.current_experience].dataset
    print('This task contains', len(current_test_set), 'test examples')
    test_ds.append(current_test_set)
    print(80*'-')

# Getting a Sample from the first Task
batch_gen = torch.utils.data.DataLoader(train_ds[0], 
                              batch_size=9, 
                              shuffle=True, 
                              num_workers=1,
                              )
for batch in tqdm(batch_gen):
    sample = batch[0]
    target = batch[1]
    print(sample.shape)
    break

Files already downloaded and verified
Files already downloaded and verified
Start of task  0 experience.current_experience 0
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
Task 0
This task contains 3000 training examples
This task contains 2864 test examples
--------------------------------------------------------------------------------
Start of task  0 experience.current_experience 1
Classes in this task: [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124]
Task 0
This task contains 749 training examples
This task contains 714 test examples
------------

  0%|                                                                                                                             | 0/334 [00:00<?, ?it/s]

torch.Size([9, 3, 128, 128])


  0%|                                                                                                                             | 0/334 [00:00<?, ?it/s]


In [9]:
tasks = [i for i in range(n_split_experiences)]
acc_by_task = {i: 0 for i in range(n_split_experiences)}
results_diff_models = {}
best_model = {}
cms = {}
patience = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

for loop in range(5):
    #loop=1
    if os.path.isfile('stop.txt'):
        print(f'You should continue from loop {loop}')
        break

    config = {
        "learning_rate": 0.00001,
        "weight_decay": 5.e-4,
        "batch_size": 8,
        "optimizer": "Adam",
        "architecture": "VGG19",
        "dataset": f'{dataset_name.upper()}',
        "epochs": 300,
        "lr_schedule": "ReduceLROnPlateau - Patience 10 - Monitoring Val Accuracy",
        "description": "Testing CUBs200 splited into 5 tasks."
    }
    wandb.init(
        # set the wandb project where this run will be logged
        project=f"cf-lora-{dataset_name}-baseline-all-parameters",
        name=f"vgg19-lr-00001-5tasks-lora-{loop}",
        
        # track hyperparameters and run metadata
        config=config
    )
    
    for model_type in ['vgg19']:
        bz=config['batch_size']
            
        torch.backends.cudnn.benchmark = False
        torch.manual_seed(0)
        random.seed(0)
        np.random.seed(0)
                       
        for task in tasks:
            model = models.vgg19_bn(weights="IMAGENET1K_V1")
            model.classifier[6] = nn.Linear(4096, total_number_classes)
            
            epoch_losses = {'train_acc': [], 'train_loss': [], 'test_acc': [], 'test_loss': []}
            model.to(device)
            print(40*'-', f'TASK_{task}', 40*'-')
        
            criterion = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
            lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=4, verbose=True)
            
            early_stop = 0
            best_val = 0
            for i in range(config['epochs']):
                g = torch.Generator()
                g.manual_seed(0)
                batch_gen = torch.utils.data.DataLoader(train_ds[task], 
                                                      batch_size=config['batch_size'], 
                                                      shuffle=True, 
                                                      num_workers=1,
                                                      )
                print(f'Training task {task} in epoch {i}. Batch size: {bz}.')
                count = 0
                total_loss = .0
                total_hit = 0
                for batch in tqdm(batch_gen):
                    sample = batch[0].to(device)
                    target = batch[1]
                    target_onehot = F.one_hot(target, num_classes=total_number_classes).to(torch.float).to(device)
    
                    y_hat = model(sample)
        
                    # Compute the loss
                    loss_training = criterion(y_hat, target_onehot)            
                    
                    # Backpropagation and optimization
                    optimizer.zero_grad()
                    loss_training.backward()
                    optimizer.step()
                    
                    total_loss += loss_training
                    total_hit += sum(np.argmax(y_hat.cpu().detach().numpy(), axis=1) == target.numpy())
    
                    sample.to('cpu')
                    target_onehot.to('cpu')
                    del sample, target_onehot
                    gc.collect()
                    torch.cuda.empty_cache()
                    
                # Evaluate in test DS after each epoch
                with torch.no_grad():
                    batch_gen_test = torch.utils.data.DataLoader(test_ds[task], 
                                                      batch_size=config['batch_size'], 
                                                      shuffle=True, 
                                                      num_workers=1,
                                                      )
                    model.eval()
                    test_loss = 0.
                    acc = 0.
                    for batch_test in tqdm(batch_gen_test):
                        sample = batch_test[0].to(device)
                        target = batch_test[1]
                        target_onehot = F.one_hot(target, num_classes=total_number_classes).to(torch.float).to(device)
            
                        y_hat = model(sample)
                        cpu_inference = y_hat.argmax(axis=1)
                        
                        test_loss += criterion(y_hat, target_onehot)
                        acc += sum(cpu_inference.cpu() == target).item()
    
                        sample.to('cpu')
                        target_onehot.to('cpu')
                        del sample, target_onehot
                        gc.collect()
                        torch.cuda.empty_cache()
    
                    test_loss = (test_loss/len(batch_gen_test)).cpu()
                    acc = acc/len(test_ds[task])
                    lr_scheduler.step(acc)
                    model.train()
                
                epoch_losses['train_acc'].append(total_hit/(len(batch_gen)*bz))
                epoch_losses['train_loss'].append((total_loss/len(batch_gen)).cpu().item())
                
                epoch_losses['test_acc'].append(acc)
                epoch_losses['test_loss'].append(test_loss.item())
        
                # if acc > (best_val+0.01):
                if acc > (best_val):
                    model.to('cpu')
                    best_model[f'{model_type}-{task}'] = deepcopy(model)
                    model.to(device)
                    best_val = acc
                    early_stop = 0
        
                if early_stop > patience:
                    break
                
                early_stop += 1
    
                wandb.log({"train_acc": (total_hit/(len(batch_gen)*bz)), "train_loss": (total_loss/len(batch_gen)), 
                           "val_acc": acc, "val_loss": test_loss, "task": (task+1),
                           "Accuracy": acc, "Loss": test_loss, 
                           "patience": early_stop, "best_val_acc": best_val, "learning_rate": optimizer.param_groups[0]['lr']
                          })
                
                print(f'Trainig acc: {total_hit/(len(batch_gen)*bz):.4}   //   Training loss: {(total_loss/len(batch_gen)):.4f}   //   Test acc: {acc:.4f}   //   Test loss: {test_loss:.4f}')
                print(f'early_stop: {early_stop}  /   Best acc: {best_val}')
                del batch_gen, batch_gen_test
                gc.collect()
                torch.cuda.empty_cache()
    
            model.to('cpu')
            del model
            model = best_model[f'{model_type}-{task}']
            del best_model[f'{model_type}-{task}']
    
            gc.collect()
            torch.cuda.empty_cache()
            
            # del criterion, optimizer, lr_scheduler 
            gc.collect()
            torch.cuda.empty_cache()
    
        wandb.finish()

/home/everton_aleixo/Documents/mahle/server_python/venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


---------------------------------------- TASK_0 ----------------------------------------
Training task 0 in epoch 0. Batch size: 8.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 358/358 [00:42<00:00,  8.48it/s]


Trainig acc: 0.07067   //   Training loss: 4.5307   //   Test acc: 0.3820   //   Test loss: 2.8737
early_stop: 1  /   Best acc: 0.3819832402234637
Training task 0 in epoch 1. Batch size: 8.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 358/358 [00:44<00:00,  8.12it/s]


...

Trainig acc: 0.984   //   Training loss: 0.0552   //   Test acc: 0.8131   //   Test loss: 0.6788
early_stop: 10  /   Best acc: 0.8321964529331515
Training task 4 in epoch 36. Batch size: 8.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 92/92 [00:09<00:00,  9.21it/s]


Trainig acc: 0.9907   //   Training loss: 0.0278   //   Test acc: 0.8186   //   Test loss: 0.6898
early_stop: 11  /   Best acc: 0.8321964529331515
Training task 4 in epoch 37. Batch size: 8.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 92/92 [00:10<00:00,  8.73it/s]


Accuracy,▅▇▇▇▇▇▇▇▇▇▇▇▇▁▆▇▇▇▇▄▆▆▆▇▆▇▄▇▇▇▇▇▅▇██████
Loss,▄▂▂▂▂▂▂▂▂▂▂▂▂█▂▂▂▂▂▅▂▂▂▂▃▂▅▂▂▂▂▂▄▁▁▁▁▁▁▁
best_val_acc,▅▇▇▇▇▇▇▇▇▇▇▇▇▁▆▇▇▇▇▄▆▆▆▇▇▇▄▇▇▇▇▇▅▇██████
learning_rate,████████▂▂▂▁▁█████▂██████▂█████▂█████▁▁▁
patience,▁▁▁▂▂▂▃▂▅▂▃▂▆▁▁▂▁▄▇▁▁▁▁▁▃▆▁▁▁▂▂▇▁▁▁▁▄█▃▇
task,▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▅▅▅▅▅▅▅▆▆▆▆▆▆████████
train_acc,▃▆▇██████████▁▇████▂▆▇████▃▆████▃▆██████
train_loss,▅▂▂▁▁▁▁▁▁▁▁▁▁█▂▁▁▁▁▅▂▁▁▁▁▁▄▂▁▁▁▁▅▂▁▁▁▁▁▁
val_acc,▅▇▇▇▇▇▇▇▇▇▇▇▇▁▆▇▇▇▇▄▆▆▆▇▆▇▄▇▇▇▇▇▅▇██████
val_loss,▄▂▂▂▂▂▂▂▂▂▂▂▂█▂▂▂▂▂▅▂▂▂▂▃▂▅▂▂▂▂▂▄▁▁▁▁▁▁▁
Accuracy,0.81855
